# Fouille données

In [ ]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from collections import Counter

%matplotlib inline
sns.set(style="whitegrid")

# Importation des données

In [9]:
df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")
print("Nombre de photos au départ :  " + str(len(df)))

Nombre de photos au départ :  420240


C:\Users\nputo\AppData\Local\Temp\ipykernel_27496\2839054439.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")


# Nettoyage des données

In [10]:
df.columns = df.columns.str.strip()

df_clean = df.dropna(axis=1, how='all')

cols_date = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']

df_clean = df_clean.drop_duplicates(subset=['user', 'lat', 'long'] + cols_date)
df_clean = df_clean.dropna(subset=cols_date)

for col in cols_date:
    df_clean[col] = df_clean[col].astype(int)

colonnes_utiles = ['id', 'user', 'lat', 'long', 'tags', 'title'] + cols_date
df_clean = df_clean[colonnes_utiles]

print("\nDonnées nettoyées :")
display(df_clean.head())
print("Nombre de photos restantes : " + str(len(df_clean)))

pourcentage_supprimes = ((len(df) - len(df_clean)) / len(df)) * 100
print(f"Pourcentage de données supprimées : {pourcentage_supprimes:.2f}%")


Données nettoyées :


,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10


Nombre de photos restantes : 131973
Pourcentage de données supprimées : 68.60%


In [ ]:
print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")

df_clean = df_clean[
    (df_clean['date_taken_year'] >= 2004) &
    (df_clean['date_taken_year'] <= 2026)
]

print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")
print("Après filtre temporel : " + str(len(df_clean)) + " photos.")

In [ ]:
# Vérifier que les coordonnées géographiques sont cohérentes.
photos_avant_filtre = len(df_clean)

df_clean = df_clean[
    (df_clean['lat'] >= -90) & (df_clean['lat'] <= 90) &
    (df_clean['long'] >= -180) & (df_clean['long'] <= 180)
]
photos_supprimees = photos_avant_filtre - len(df_clean)
print(f"Photos supprimées par le filtre géographique : {photos_supprimees}")
print(f"Photos avec coordonnées valides : {len(df_clean)}")
print(f"Min/Max Latitude  : {df_clean['lat'].min()} / {df_clean['lat'].max()}")
print(f"Min/Max Longitude : {df_clean['long'].min()} / {df_clean['long'].max()}")

In [28]:
from folium.plugins import FastMarkerCluster
# Création de la carte centrée sur une latitude et une longitude moyennes
m = folium.Map(location=[45.7640, 4.8357], zoom_start=13)
locations = list(zip(df['lat'], df['long']))
FastMarkerCluster(data=locations).add_to(m)
# Sauvegarde la carte dans un fichier HTML
m.save('ma_carte_lyon_old.html')
 

# Clustering

In [15]:
df_clustering = df_clean[['lat', 'long']]

In [19]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering)

# show
scaled_data_df = pd.DataFrame(data=scaled_data, columns=['lat', 'long'])

print("Aperçu des données standardisées (prêtes pour l'algo) :")
display(scaled_data_df.head())

print(f"Moyennes : \n{scaled_data_df.mean()}")

Aperçu des données standardisées (prêtes pour l'algo) :


,lat,long
0,-0.463615,-0.583102
1,-0.523467,0.851302
2,-0.245126,0.281303
3,0.634748,1.238013
4,0.634748,1.238013


Moyennes : 
lat    -2.054647e-13
long   -2.199138e-14
dtype: float64


In [ ]:
# Initialisation de la liste pour stocker les inerties
inertia = []
k_range = range(1, 100)  # On teste de 1 à 10 clusters

# Boucle pour calculer l'inertie pour chaque nombre de clusters
for k in k_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(scaled_data_df)
    inertia.append(kmeans.inertia_)

# Création du graphique (Plot)
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_range)
plt.grid(True)
plt.show()

KeyboardInterrupt: 

Au vu des résultats, nous pouvons partir sur une valeur entre 15 et 20. Cela permet d'avoir suffisant de zones d'intérêt pour une grande ville comme Lyon, tout en évitant de diviser les zones d'intéret en plusieurs cluster. Choisisons donc une valeur de 17 clusters.

In [67]:
kmeans = KMeans(n_clusters=500, init='k-means++', random_state=42)

kmeans.fit(scaled_data_df)

df_clean['cluster'] = kmeans.labels_

df_clean.head()

,id,user,lat,long,tags,title,date_taken_year,date_taken_month,date_taken_day,date_taken_hour,date_taken_minute,cluster kmeans,cluster
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,2010,2,28,15,11,9,363
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,2010,2,28,17,51,3,286
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,2010,2,28,17,29,3,142
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,2010,1,28,20,15,10,121
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,2010,1,28,20,10,10,121


In [ ]:
import folium
from scipy.spatial import ConvexHull
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

counts = df_clean[df_clean['cluster'] != -1]['cluster'].value_counts()
min_val = counts.min()
max_val = counts.max()

print(f"Cluster min : {min_val} photos | Cluster max : {max_val} photos")

colors = ["white", "darkred"] 
cmap = mcolors.LinearSegmentedColormap.from_list("mon_degrade_bleu_rouge", colors)
norm = mcolors.LogNorm(vmin=min_val, vmax=max_val)

def get_color(n):
    return mcolors.to_hex(cmap(norm(n)))

# ==========================================
# 2. GÉNÉRATION DE LA CARTE
# ==========================================

m = folium.Map(location=[df_clean['lat'].mean(), df_clean['long'].mean()], zoom_start=13)

clusters_ids = sorted(df_clean['cluster'].unique())

for cluster_id in clusters_ids:
    if cluster_id == -1: continue # On saute le bruit

    # Données du cluster
    points = df_clean[df_clean['cluster'] == cluster_id][['lat', 'long']].values
    n_photos = len(points)
    
    # Récupération de la couleur interpolée
    color = get_color(n_photos)
    
    # Plus c'est important, plus c'est opaque (de 0.3 à 0.8)
    opacity = 0.3 + (norm(n_photos) * 0.5)

    # DESSIN DES FORMES
    if len(points) >= 3:
        try:
            hull = ConvexHull(points)
            points_contour = points[hull.vertices]
            
            folium.Polygon(
                locations=points_contour,
                color=color,
                weight=2,
                fill=True,
                fill_color=color,
                fill_opacity=opacity,
                popup=f"Zone {cluster_id}: {n_photos} photos"
            ).add_to(m)
            
            # Label au centre
            center = np.mean(points, axis=0)
            # J'ai mis le texte en blanc avec contour noir pour que ce soit lisible sur le rouge et le bleu
            style_texte = "color: white; text-shadow: 1px 1px 2px black; font-weight: bold; font-size: 10pt;"
            folium.Marker(
                center,
                icon=folium.DivIcon(html=f'<div style="{style_texte}">{n_photos}</div>')
            ).add_to(m)
            
        except:
            # Fallback si forme impossible
            pass
    else:
        # Petits points pour les tout petits clusters
        for pt in points:
            folium.CircleMarker(pt, radius=5, color=color, fill=True, fill_color=color).add_to(m)

# Export
nom_fichier = 'carte_degrade_bleu_rouge.html'
m.save(nom_fichier)
print(f"✅ Carte générée : {nom_fichier}")

Cluster min : 4 photos | Cluster max : 7963 photos
✅ Carte générée : carte_degrade_bleu_rouge.html
